In [8]:
# OpenAI API Examples Notebook
# 
# This notebook demonstrates key features of the OpenAI API for developers.
# It provides practical examples of the most important capabilities with executable code.

# First, let's set up our environment by installing the OpenAI Python library
%pip install openai numpy matplotlib pydantic --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import the OpenAI library
import os
from openai import OpenAI

In [ ]:
MODEL = "gpt-4.1-mini"

In [5]:
# Initialize the client
# Note: In a real application, you would use an environment variable or secure method
# to store your API key. This is just for demonstration.
client = OpenAI(
    # Replace with your actual API key or use: api_key=os.environ.get("OPENAI_API_KEY")
    api_key="YOUR_API_KEY_HERE"
)

## 1. Basic Text Generation

In [7]:
prompt = "Write a one-sentence bedtime story about a unicorn."

response = client.responses.create(
    model=MODEL,
    input=prompt
)

print(f"Prompt: {prompt}")
print(f"Response: {response.output_text}")

# Show how to control generation with parameters
print("\nWith controlled parameters:")
response = client.responses.create(
    model=MODEL,
    input=prompt,
    temperature=0.7,  # Lower for more deterministic outputs
    top_p=0.9         # Nucleus sampling
)

print(f"Response (controlled): {response.output_text}")

Prompt: Write a one-sentence bedtime story about a unicorn.
Response: Under a silvery moon, a gentle unicorn named Luna danced through a field of enchanted flowers, spreading dreams of magic to all the sleeping children nearby.

With controlled parameters:
Response (controlled): Under a blanket of twinkling stars, a gentle unicorn named Luna danced through a moonlit meadow, leaving trails of shimmering dreams for children to find as they drifted off to sleep.


---

## 2. Structured Outputs

In [1]:
import json
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
   model=MODEL,
    input=[
        {"role": "system", "content": "You are a UI generator AI. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "ui",
            "description": "Dynamically generated UI",
            "schema": {
                "type": "object",
                "properties": {
                    "type": {
                        "type": "string",
                        "description": "The type of the UI component",
                        "enum": ["div", "button", "header", "section", "field", "form"]
                    },
                    "label": {
                        "type": "string",
                        "description": "The label of the UI component, used for buttons or form fields"
                    },
                    "children": {
                        "type": "array",
                        "description": "Nested UI components",
                        "items": {"$ref": "#"}
                    },
                    "attributes": {
                        "type": "array",
                        "description": "Arbitrary attributes for the UI component, suitable for any element",
                        "items": {
                            "type": "object",
                            "properties": {
                              "name": {
                                  "type": "string",
                                  "description": "The name of the attribute, for example onClick or className"
                              },
                              "value": {
                                  "type": "string",
                                  "description": "The value of the attribute"
                              }
                          },
                          "required": ["name", "value"],
                          "additionalProperties": False
                      }
                    }
                },
                "required": ["type", "label", "children", "attributes"],
                "additionalProperties": False
            },
            "strict": True,
        },
    },
)

ui = json.loads(response.output_text)


---

## 2.1 Structured Outputs with Pydantic models

In [3]:
from pydantic import BaseModel

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

steps=[Step(explanation='Start by subtracting 7 from both sides to isolate the term with x.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides; 7 - 7 is 0, and -23 - 7 is -30.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='8x / 8 = -30 / 8'), Step(explanation='Simplify -30 divided by 8 to get x alone. You can reduce this fraction by dividing numerator and denominator by 2.', output='x = -15/4')] final_answer='x = -15/4'


---

## 3. Multimodal Capabilities - Vision

In [5]:
# For notebook demonstration, we'll use a placeholder URL
image_url = "https://images.unsplash.com/photo-1579546929518-9e396f3cc809?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxleHBsb3JlLWZlZWR8MXx8fGVufDB8fHx8&w=1000&q=80"

response = client.responses.create(
    model=MODEL,
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": image_url,
            },
        ],
    }],
)

print(response.output_text)


This image shows a smooth gradient of colors blending into each other. The colors include shades of red, pink, orange, yellow, green, blue, and purple, creating a soft and visually pleasing transition across the image. There are no distinct shapes or objects visible; it is purely an abstract color gradient.


---

## 4. Audio Capabilities - Text to Speech

In [8]:
speech_file_path = "speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build something people love!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

---

## 4.1 Audio Capabilities - Speech to Text

In [9]:
audio_file= open("speech.mp3", "rb")

transcription = client.audio.transcriptions.create(
    model="gpt-4o-transcribe", 
    file=audio_file
)

print(transcription.text)

Today is a wonderful day to build something people love.


---

## 5. Function Calling

In [2]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)

In [3]:
# Extract the tool call and arguments
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
# Call the function
result = get_weather(args["latitude"], args["longitude"])

In [ ]:
# Append the tool call and result to the input messages
input_messages.append(tool_call) # append model's function call message
input_messages.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in Paris today is approximately 13.8°C. If you need more details like conditions (sunny, rainy, etc.), feel free to specify!


---

## 6. Reasoning Models

In [6]:
# response_o1 = client.responses.create(
#     model="o1",
#     input="Design an algorithm to find the shortest path in a graph."
# )

# print("o1 Response:")
# print(response_o1.output_text)

# Using o4-mini for faster reasoning
response_o4 = client.responses.create(
    model="o4-mini",
    input="Explain how to implement a hash table."
)

print("\no3-mini Response:")
print(response_o4.output_text)


o3-mini Response:
Here’s a step-by‑step recipe for implementing a basic hash table with separate chaining (i.e. each slot holds a linked list of key‑value pairs). You can adapt this to your favorite language.

1. Choose your core parameters  
   • Table size N (preferably a prime or power of two)  
   • A hash function h(key) that maps keys to integers in [0..N–1]  

2. Define your data structures  
   • Entry { key, value, next }   // for a singly linked list node  
   • HashTable { buckets: array of Entry pointers of length N, count }  

3. Initialize  
   • Allocate the buckets array of size N, initialize each bucket to null  
   • Set count = 0  

4. Hash function  
   • If keys are integers: h(k) = (a·k + b) mod N  
   • If strings: e.g. compute a rolling polynomial hash over characters, then mod N  
   • Always reduce the result to [0..N–1]  

5. Insert(key, value)  
   idx = h(key)  
   node = buckets[idx]  
   while node ≠ null:  
     if node.key == key:  
       node.value =

In [7]:
## With reasoning models, you can view the reasoning process and the steps taken to arrive at the answer.
print("This is the reasoning configuration for o4-mini:", response_o4.reasoning)

This is the reasoning configuration for o3-mini: Reasoning(effort='medium', generate_summary=None, summary=None)


## 7. Embeddings

In [8]:
import numpy as np

# First, let's create embeddings for a set of words
words = [
    "king", "queen", "man", "woman", 
    "apple", "banana", "orange", "pear",
    "castle", "throne"
]

# Get embeddings for all words
response = client.embeddings.create(
    model="text-embedding-3-large",
    input=words,
    encoding_format="float"
)

# Extract the embeddings
embeddings = [data.embedding for data in response.data]

print(f"Embedding dimension: {len(embeddings[0])}")
print(f"Number of embeddings: {len(embeddings)}")

# Function to compute dot product between two vectors
import numpy as np

def dot_product(vec1, vec2):
    return np.dot(vec1, vec2)

# Compute similarity matrix (dot products between all pairs)
similarity_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        similarity_matrix[i][j] = dot_product(embeddings[i], embeddings[j])

# Print similarity matrix with labels
print("\nSimilarity Matrix (Dot Products):")
print("          " + " ".join(f"{word:<8}" for word in words))
for i, word in enumerate(words):
    row_values = " ".join(f"{similarity_matrix[i][j]:.4f}  " for j in range(len(words)))
    print(f"{word:<10} {row_values}")

# Check specific relationships
king_queen_similarity = dot_product(embeddings[0], embeddings[1])
apple_banana_similarity = dot_product(embeddings[4], embeddings[5])

print("\nSpecific relationships:")
print(f"Similarity between 'king' and 'queen': {king_queen_similarity:.4f}")
print(f"Similarity between 'apple' and 'banana': {apple_banana_similarity:.4f}")
print(f"Similarity between 'king' and 'apple': {dot_product(embeddings[0], embeddings[4]):.4f}")

# We expect king/queen to be closer to each other than king/apple
# And apple/banana to be closer to each other than queen/banana


Embedding dimension: 3072
Number of embeddings: 10

Similarity Matrix (Dot Products):
          king     queen    man      woman    apple    banana   orange   pear     castle   throne  
king       1.0000   0.5552   0.4183   0.2938   0.3243   0.3305   0.2879   0.2802   0.3615   0.4027  
queen      0.5552   1.0000   0.3072   0.4132   0.3145   0.3191   0.2983   0.2996   0.2969   0.3354  
man        0.4183   0.3072   1.0000   0.5713   0.3098   0.3495   0.2972   0.2695   0.2998   0.2650  
woman      0.2938   0.4132   0.5713   1.0000   0.3199   0.2937   0.2784   0.2533   0.2449   0.2491  
apple      0.3243   0.3145   0.3098   0.3199   1.0000   0.4619   0.4588   0.4391   0.3002   0.2340  
banana     0.3305   0.3191   0.3495   0.2937   0.4619   1.0000   0.4579   0.3636   0.2777   0.2075  
orange     0.2879   0.2983   0.2972   0.2784   0.4588   0.4579   1.0000   0.3822   0.2848   0.2174  
pear       0.2802   0.2996   0.2695   0.2533   0.4391   0.3636   0.3822   1.0000   0.2788   0.1929  
castle